In [9]:
import mne
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator, make_inverse_operator, apply_inverse
from mne.viz import circular_layout
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_connectivity_circle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import glob

In [2]:
# read one sample data
mne_filename = "mne_data/C10_32Ch_48Subjects_al_raw_eeg.fif"
raw = mne.io.read_raw_fif(mne_filename, verbose=False)

In [3]:
events = mne.make_fixed_length_events(raw, start=0, stop=20, duration=2., overlap=1.75, id = 1)

In [4]:
# Download fsaverage files
fs_dir = mne.datasets.fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)

# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

0 files missing from root.txt in C:\Users\work\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage


In [5]:
# Clean channel names to be able to use a standard 1005 montage
new_names = dict(
    (ch_name,
     ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
    for ch_name in raw.ch_names)
raw.rename_channels(new_names)

# Read and set the EEG electrode locations, which are already in fsaverage's
# space (MNI space) for standard_1020:
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)
raw.set_eeg_reference(projection=True)  # needed for inverse modeling

# Check that the locations of EEG electrodes is correct with respect to MRI
# fig = mne.viz.plot_alignment(
#     raw.info, eeg=['original', 'projected'], src=src, trans=trans, subject=subject, bem=bem, surfaces=['head'],
#     mri_fiducials='estimated', show_axes=True)

# fig.savefig('coregistration.png', dpi=300)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated


<Raw | C10_32Ch_48Subjects_al_raw_eeg.fif, 32 x 5120 (20.0 s), ~54 kB, data not loaded>

In [6]:
src_orig = mne.read_source_spaces(src)

    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read


In [7]:
fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                                bem=bem, eeg=True, mindist=5.0)
print(fwd)

Source space          : C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif
MRI -> head transform : c:\Users\work\Anaconda3\envs\kia\lib\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif
Accurate field computations
Do computations in head coordinates
Free source orientations

Reading C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage\bem\fsaverage-ico-5-src.fif...
Read 2 source spaces a total of 20484 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999994  0.003552  0.000202      -1.76 mm
    -0.003558  0.998389  0.056626      31.09 mm
    -0.000001 -0.056626  0.998395      39.60 mm
     0.000000  0.000000  0.000000       1.00

Read  32 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Setting up the BEM mode

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 7809 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 11444.8 ms
Checking surface interior status for 10242 points...
    Found  2241/10242 points inside  an interior sphere of radius   47.7 mm
    Found     0/10242 points outside an exterior sphere of radius   98.3 mm
    Found     0/ 8001 points outside using surface Qhull


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found     0/ 8001 points outside using solid angles
    Total 10242/10242 points inside the surface
Interior check completed in 7055.0 ms



[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.9s finished


Setting up for EEG...
Computing EEG at 20484 source locations (free orientations)...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.



Finished.
<Forward | MEG channels: 0 | EEG channels: 32 | Source space: Surface with 20484 vertices | Source orientation: Free>


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s finished


In [8]:
epochs = mne.make_fixed_length_epochs(raw=raw, duration=2, overlap=0.75)

Not setting metadata
15 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated


In [9]:
noise_cov = mne.compute_covariance(epochs, tmax=2)

inverse_operator = make_inverse_operator(epochs.info, fwd, noise_cov,
                                         loose=0.2, depth=0.8)

Loading data for 15 events and 512 original time points ...
0 bad epochs dropped
Computing rank from data with rank=None
    Using tolerance 9.8e-06 (2.2e-16 eps * 32 dim * 1.4e+09  max singular value)
    Estimated rank (eeg): 31
    EEG: rank 31 computed from 32 data channels with 1 projector
    Created an SSP operator (subspace dimension = 1)


C:\Users\work\AppData\Local\Temp/ipykernel_7768/406830044.py:1: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  noise_cov = mne.compute_covariance(epochs, tmax=2)


    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 32 -> 31
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 7680
[done]
Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 32 channels.
    32 out of 32 channels remain after picking
Selected 32 channels
Creating the depth weighting matrix...
    32 EEG channels
    limit = 20485/20484 = 2.518446
    scale = 60426.8 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 0.018 (2.2e-16 eps * 32 dim * 2.5e+12  max singular value)
    Estimated rank (eeg): 24
    EEG: rank 24 computed from 32 data 

In [10]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stcs = apply_inverse_epochs(epochs, inverse_operator, lambda2, method,
                            pick_ori="normal", return_generator=True)

In [13]:
# Get labels for FreeSurfer 'aparc' cortical parcellation with 34 labels/hemi
labels = mne.read_labels_from_annot('fsaverage', parc='aparc',
                                    subjects_dir=subjects_dir)
labels.pop(-1)
label_colors = [label.color for label in labels]

# Average the source estimates within each label using sign-flips to reduce
# signal cancellations, also here we return a generator
src = inverse_operator['src']
label_ts = mne.extract_label_time_course(
    stcs, labels, src, mode='mean_flip', return_generator=True)

fmin = (0.5, 4., 8., 13., 30.)
fmax = (4., 8., 13., 30., 45.)
sfreq = raw.info['sfreq']  # the sampling frequency
con_methods = ['dpli']
con = spectral_connectivity_epochs(
    label_ts, method="dpli", mode='multitaper', sfreq=sfreq, fmin=fmin,
    fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)

Reading labels from parcellation...
   read 35 labels from C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage\label\lh.aparc.annot
   read 34 labels from C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage\label\rh.aparc.annot
Connectivity computation...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 24 (8 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Picked 32 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 15
Extracting time courses for 68 labels (mode: mean_flip)
only using indices for lower-triangular matrix
    computing connectivity for 2278 connections
    using t=0.000s..1.996s for estimation (512 points)
    computing connectivity for the bands:
     band 1: 0.5Hz..4

C:\Users\work\AppData\Local\Temp/ipykernel_7768/874600875.py:19: RuntimeWarning: fmin=0.500 Hz corresponds to 1.000 < 5 cycles based on the epoch length 2.000 sec, need at least 10.000 sec epochs or fmin=2.500. Spectrum estimate will be unreliable.
  fmax=fmax, faverage=True, mt_adaptive=True, n_jobs=1)


Processing epoch : 2 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
Processing epoch : 9 / 15
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 9
Processing epoch : 10 / 15
Extra

In [14]:
con_data = con.get_data(output='dense')

In [15]:
new_con_data = np.swapaxes(np.swapaxes(con_data, 0, 2), 1, 2)

In [16]:
fs_dir = mne.datasets.fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)

labels = mne.read_labels_from_annot('fsaverage', parc='aparc',
                                            subjects_dir=subjects_dir, verbose=False)
labels.pop(-1) # remove the unknown label
label_names = [label.name for label in labels]

0 files missing from root.txt in C:\Users\work\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage


In [34]:
# for data in new_con_data:
data = new_con_data[0]
df_corr = pd.DataFrame(data, columns = label_names, index = label_names)
df_pair = df_corr.stack().reset_index()
df_pair.columns = ["source", "target", "value"]

In [35]:
df_pair

,source,target,value
0,bankssts-lh,bankssts-lh,0.000000
1,bankssts-lh,bankssts-rh,0.000000
2,bankssts-lh,caudalanteriorcingulate-lh,0.000000
3,bankssts-lh,caudalanteriorcingulate-rh,0.000000
4,bankssts-lh,caudalmiddlefrontal-lh,0.000000
...,...,...,...
4619,transversetemporal-rh,supramarginal-rh,0.408333
4620,transversetemporal-rh,temporalpole-lh,0.341667
4621,transversetemporal-rh,temporalpole-rh,0.608333
4622,transversetemporal-rh,transversetemporal-lh,0.575000


In [36]:
# for r in range(len(df_pair)):
#     source = df_pair.iloc[r]["source"]
#     target = df_pair.iloc[r]["target"]
#     value = df_pair.iloc[r]["value"]
#     # if value > 0.5, set value of cell to 0 and set value of cell with source and target swapped to the value
#     if value >= 0.5:
#         df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = 0
#         df_pair.at[df_pair.index[(df_pair['source'] == target) & (df_pair['target'] == source)], 'value'] = (value - 0.5)/0.5
#     elif value > 0 and value < 0.5:
#         df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = (0.5 - value)/0.5

In [37]:
for r in range(len(df_pair)):
    source = df_pair.iloc[r]["source"]
    target = df_pair.iloc[r]["target"]
    value = df_pair.iloc[r]["value"]
    # if value > 0.5, set value of cell to 0 and set value of cell with source and target swapped to the value
    if value >= 0.6:
        df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = -(value - 0.5)/0.5
    elif value > 0 and value < 0.4:
        df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = (0.5 - value)/0.5
    else:
        df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = 0

In [38]:
df_good = df_pair[df_pair['value'] != 0]
# df_good = df_pair

In [39]:
df_good

,source,target,value
68,bankssts-rh,bankssts-lh,0.283333
137,caudalanteriorcingulate-lh,bankssts-rh,0.450000
205,caudalanteriorcingulate-rh,bankssts-rh,0.333333
206,caudalanteriorcingulate-rh,caudalanteriorcingulate-lh,-0.600000
342,caudalmiddlefrontal-rh,caudalanteriorcingulate-lh,0.483333
...,...,...,...
4614,transversetemporal-rh,superiorparietal-lh,0.350000
4615,transversetemporal-rh,superiorparietal-rh,-0.250000
4616,transversetemporal-rh,superiortemporal-lh,0.233333
4620,transversetemporal-rh,temporalpole-lh,0.316667


In [2]:
lobe_map = {"Frontal":{
                "frontalpole":"FP",
                "medialorbitofrontal":"MO",
                "lateralorbitofrontal":"LO",
                "paracentral":"ParaC",
                "parsopercularis":"POp",
                "parsorbitalis":"POr",
                "parstriangularis":"PT",
                "precentral": "PreC",
                "rostralanteriorcingulate":"RAC",
                "rostralmiddlefrontal":"RMF",
                "superiorfrontal":"SF",
                "caudalanteriorcingulate":"CACC",
                "caudalmiddlefrontal":"CMF",
            },
            "Parietal":{
                "superiorparietal":"SP",
                "inferiorparietal":"IP",
                "isthmuscingulate":"IC",
                "postcentral":"PG",
                "posteriorcingulate":"PCC",
                "precuneus":"PCUN",
                "supramarginal":"SG",
            },
            "Temporal":{
                "bankssts":"BSTS",
                "entorhinal":"ENT",
                "fusiform":"FUS",
                "middletemporal":"MT",
                "parahippocampal":"PHG",
                "superiortemporal": "STG",
                "temporalpole":"TP",
                "transversetemporal":"TTG",
                "inferiortemporal":"ITG",
            },
            "Occipital":{
                "cuneus":"CUN",
                "lateraloccipital":"LOC",
                "lingual":"LNG",
                "pericalcarine":"PCAL",
            },
            "Insula":{
                "insula":"INS"
            }
}
lobe_map_color = {"Frontal": (1,0,0),
                  "Insula": (0,1,0),
                "Temporal": (0,0,1),
                "Parietal": (1, 1,0), 
                 "Occipital": (0,1,1),
}

def get_lobe(name):
    # get hemisphere and label
    label = name.split("-")[0]
    hemisphere = name.split("-")[1]

    for lobe in lobe_map:
        if label in lobe_map[lobe]:
            return lobe + "-" + hemisphere
    return None

def feature_extraction_lobe_level(data):    
    # get the lobe level using the lobe_map
    data["source_lobe"] = data["source"].apply(lambda x: get_lobe(x))
    data["target_lobe"] = data["target"].apply(lambda x: get_lobe(x))
    # average the value for the same lobe
    data = data.groupby(["source_lobe", "target_lobe"]).mean().reset_index()
    # get the unique lobe
    unique_lobe = data["source_lobe"].unique()
    corr_matrix = pd.DataFrame(np.zeros((len(unique_lobe), len(unique_lobe))), columns=unique_lobe, index=unique_lobe)

    # fill the correlation matrix
    for r in range(len(data)):
        source_lobe = data.iloc[r]["source_lobe"]
        target_lobe = data.iloc[r]["target_lobe"]
        value = data.iloc[r]["value"]
        corr_matrix.at[source_lobe, target_lobe] = value

    #corr_matrix to pandas dataframe source_lobe target_lobe value
    corr_df = corr_matrix.stack().reset_index()
    corr_df = corr_df.rename(columns={"level_0": "source_lobe", "level_1": "target_lobe", 0: "value"})
    corr_df = corr_df.sort_values(by=["source_lobe", "target_lobe"])

    return corr_df

In [66]:
feature_extraction_lobe_level(df_good)

c:\Users\work\Anaconda3\envs\kia\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\Users\work\Anaconda3\envs\kia\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0        Frontal-lh_Frontal-lh
1        Frontal-lh_Frontal-rh
2         Frontal-lh_Insula-lh
3         Frontal-lh_Insula-rh
4      Frontal-lh_Occipital-lh
                ...           
95    Temporal-rh_Occipital-rh
96     Temporal-rh_Parietal-lh
97     Temporal-rh_Parietal-rh
98     Temporal-rh_Temporal-lh
99     Temporal-rh_Temporal-rh
Length: 100, dtype: object

# Run the whole data

In [10]:
mne_data = pd.read_csv("mne_data.csv")
mne_files = mne_data[["al", "fa"]].values.reshape(-1)

In [11]:
# Download fsaverage files
fs_dir = mne.datasets.fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)

# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = os.path.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = os.path.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

raw = mne.io.read_raw_fif(mne_files[0], verbose=False)

# Forward solution
# Clean channel names to be able to use a standard 1005 montage
new_names = dict(
    (ch_name,
    ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
    for ch_name in raw.ch_names)
raw.rename_channels(new_names)
# Read and set the EEG electrode locations, which are already in fsaverage's
# space (MNI space) for standard_1020:
montage = mne.channels.make_standard_montage('standard_1020')
raw.set_montage(montage)
raw.set_eeg_reference(projection=True, verbose=False)  # needed for inverse modeling

fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
                            bem=bem, eeg=True, mindist=5.0, verbose=False)

# Get labels for FreeSurfer 'aparc' cortical parcellation with 34 labels/hemi
labels = mne.read_labels_from_annot('fsaverage', parc='aparc',
                                    subjects_dir=subjects_dir, verbose=False)
labels.pop(-1) # remove the unknown label
label_colors = [label.color for label in labels]

fmin = (0.5, 4., 8., 13., 30.)
fmax = (4., 8., 13., 30., 45.)

sfreq = raw.info['sfreq']  # the sampling frequency
con_methods = ['dpli']

for mne_filename in mne_files:
    print(f"Running: {mne_filename}")
    # read file and make events
    p_id = mne_filename[10:12].replace("_","")
    if "_al_" in mne_filename: 
        condition = "al"
    elif "_fa_" in mne_filename:
        condition = "fa"
    raw = mne.io.read_raw_fif(mne_filename, verbose=False)

    # for n_raw, raw in enumerate(list_raw):
    events = mne.make_fixed_length_events(raw, start=0., duration=4., overlap=2., id = 1)
    
    # Forward solution
    # Clean channel names to be able to use a standard 1005 montage
    new_names = dict(
        (ch_name,
        ch_name.rstrip('.').upper().replace('Z', 'z').replace('FP', 'Fp'))
        for ch_name in raw.ch_names)
    raw.rename_channels(new_names)

    # Read and set the EEG electrode locations, which are already in fsaverage's
    # space (MNI space) for standard_1020:
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)
    raw.set_eeg_reference(projection=True, verbose=False)  # needed for inverse modeling
    
    # fwd = mne.make_forward_solution(raw.info, trans=trans, src=src,
    #                             bem=bem, eeg=True, mindist=5.0, verbose=False)
    
    # make epochs
    event_id = dict(epoch=1)  # event trigger and conditions
    tmin = 0  # start of each epoch
    tmax = 4  # end of each epoch
    baseline = (0, 0)  # means from the first instant to t = 0
    reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)

    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                        picks="all", baseline=baseline, verbose=False)
    
    # covariance and inverse operator
    noise_cov = mne.compute_covariance(epochs, tmax=4, verbose=False)
    inverse_operator = make_inverse_operator(epochs.info, fwd, noise_cov,
                                        loose=0.2, depth=0.8, verbose=False)
    
    method = "sLORETA"
    snr = 3.
    lambda2 = 1. / snr ** 2
    stcs = apply_inverse_epochs(epochs, inverse_operator, lambda2, method,
                                pick_ori="normal", return_generator=True, verbose=False)

    # Average the source estimates within each label using sign-flips to reduce
    # signal cancellations, also here we return a generator
    src = inverse_operator['src']
    label_ts = mne.extract_label_time_course(
        stcs, labels, src, mode='mean_flip', return_generator=True, verbose= False)
    
    con = spectral_connectivity_epochs(
        label_ts, method=con_methods, mode='multitaper', sfreq=sfreq, fmin=fmin,
        fmax=fmax, faverage=True, n_jobs=1, verbose=False)

    output_dir_prefix = "output/dpli2"

    con_values = con.get_data(output='dense')
    for i, f in enumerate(["delta", "theta", "alpha", "beta", "gamma"]):
        con_values_by_freq = con_values[:,:,i]
        np.save(f"{output_dir_prefix}/{p_id}_{condition}_{con_methods[0]}_{f}.npy", con_values_by_freq)

0 files missing from root.txt in C:\Users\work\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage
Running: mne_data/C1_32Ch_48Subjects_al_raw_eeg.fif
Connectivity computation...
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 31 (1 small eigenvalues omitted)
    Computing noise-normalization factors (sLORETA)...
[done]
Picked 32 channels from the data
Computing inverse...
    Eigenleads need to be weighted ...
Processing epoch : 1 / 9
Extracting time courses for 68 labels (mode: mean_flip)
only using indices for lower-triangular matrix
    computing connectivity for 2278 connections
    using t=0.000s..4.000s for estimation (1025 points)
    computing connectivity for the bands:
     band 1: 0.7Hz..4.0Hz (14 point

C:\Users\work\AppData\Local\Temp/ipykernel_16712/1917337658.py:100: RuntimeWarning: fmin=0.500 Hz corresponds to 2.002 < 5 cycles based on the epoch length 4.004 sec, need at least 10.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  fmax=fmax, faverage=True, n_jobs=1, verbose=False)


Processing epoch : 2 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
Processing epoch : 9 / 9
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 9
[done]
    assembling connectivity matri

C:\Users\work\AppData\Local\Temp/ipykernel_16712/1917337658.py:100: RuntimeWarning: fmin=0.500 Hz corresponds to 2.002 < 5 cycles based on the epoch length 4.004 sec, need at least 10.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  fmax=fmax, faverage=True, n_jobs=1, verbose=False)


Processing epoch : 2 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
[done]
    assembling connectivity matrix
[Connectivity computation done]
Running: mne_data/C2_32Ch_48Subjects_al_raw_eeg.fif
Connectivity computation...
Prepar

...

Processing epoch : 2 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
[done]
    assembling connectivity matrix
[Connectivity computation done]
Running: mne_data/C50_32Ch_48Subjects_al_raw_eeg.fif
Connectivity computation...
Prepa

C:\Users\work\AppData\Local\Temp/ipykernel_16712/1917337658.py:100: RuntimeWarning: fmin=0.500 Hz corresponds to 2.002 < 5 cycles based on the epoch length 4.004 sec, need at least 10.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  fmax=fmax, faverage=True, n_jobs=1, verbose=False)


Processing epoch : 2 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
[done]
    assembling connectivity matrix
[Connectivity computation done]
Running: mne_data/C50_32Ch_48Subjects_fa_raw_eeg.fif
Connectivity computation...
Prepa

C:\Users\work\AppData\Local\Temp/ipykernel_16712/1917337658.py:100: RuntimeWarning: fmin=0.500 Hz corresponds to 2.002 < 5 cycles based on the epoch length 4.004 sec, need at least 10.000 sec epochs or fmin=1.249. Spectrum estimate will be unreliable.
  fmax=fmax, faverage=True, n_jobs=1, verbose=False)


Processing epoch : 2 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 2
Processing epoch : 3 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 3
Processing epoch : 4 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 4
Processing epoch : 5 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 5
Processing epoch : 6 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 6
Processing epoch : 7 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 7
Processing epoch : 8 / 8
Extracting time courses for 68 labels (mode: mean_flip)
    computing connectivity for epoch 8
[done]
    assembling connectivity matrix
[Connectivity computation done]


# Hypothesis testing

In [12]:
# params
fs_dir = mne.datasets.fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)

labels = mne.read_labels_from_annot('fsaverage', parc='aparc',
                                            subjects_dir=subjects_dir, verbose=False)
labels.pop(-1) # remove the unknown label
label_names = [label.name for label in labels]

# all functions
# example 1_al_ciplv_theta_1.npy
def parse_filename(filename):
    s = filename.split("_")
    pId = s[0]
    label = s[1]
    method = s[2]
    freq = s[-1].split(".")[0]
    
    return {"pId": pId, "label": label, "method": method, "freq": freq}

def read_file(filename): 
    data = np.load(filename)
    return data

# get all the files in the folder output/
def get_files(folder, filter = None):
    '''
    filter is a dictionary, has 2 keys: method and freq to filter the files
    '''
    files = glob.glob(os.path.join(folder, "*.npy"))
    ret_files = []
    if filter:
        for f in files:
            # get the filename
            filename = os.path.basename(f)
            f_info = parse_filename(filename)
            
            if (f_info["method"] == filter["method"]) and (f_info["freq"] == filter["freq"]):
                ret_files.append(f)
    else:
        ret_files = files

    return ret_files

def transform_data(data):
    df_corr = pd.DataFrame(data, columns = label_names, index = label_names)
    df_pair = df_corr.stack().reset_index()
    df_pair.columns = ["source", "target", "value"]

    for r in range(len(df_pair)):
        source = df_pair.iloc[r]["source"]
        target = df_pair.iloc[r]["target"]
        value = df_pair.iloc[r]["value"]
        # if value > 0.5, set value of cell to 0 and set value of cell with source and target swapped to the value
        if value > 0.6:
            df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = -(value - 0.5)/0.5
        elif value > 0 and value < 0.4:
            df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = (0.5 - value)/0.5
        else:
            df_pair.at[df_pair.index[(df_pair['source'] == source) & (df_pair['target'] == target)], 'value'] = 0

    df_good = df_pair[df_pair['value'] != 0]

    return df_good

0 files missing from root.txt in C:\Users\work\mne_data\MNE-fsaverage-data
0 files missing from bem.txt in C:\Users\work\mne_data\MNE-fsaverage-data\fsaverage


In [13]:
freqs = ['delta', 'theta', 'alpha', 'beta', 'gamma']
method = 'dpli'
output_dir_prefix = "dpli2"

In [15]:
d_stats_df = {}
for freq in freqs:
    print(f"Processing {freq}...")
    filter = {"method": method, "freq": freq}
    files = get_files("output/dpli2/", filter)

    df_all_participants = pd.DataFrame()
    for f in files:
        print(f"Processing {f}...")
        data = read_file(f)
        file_info = parse_filename(os.path.basename(f))
        pId = file_info["pId"]
        label = file_info["label"]

        df_good = transform_data(data)
        df_features = feature_extraction_lobe_level(df_good)
        df_features["con"] = df_features[['source_lobe', 'target_lobe']].apply(lambda x: '_'.join(x), axis=1)

        con = df_features["con"].values
        val = df_features["value"].values.reshape(1, -1)
        df_x = pd.DataFrame(columns=con, data=val)
        df_x["pId"] = pId
        df_x["label"] = label

        df_all_participants = df_all_participants.append(df_x)

    df_all_participants.to_csv(f"{output_dir_prefix}/{method}_{freq}.csv", index=False)
    d_stats_df[freq] = df_all_participants


Processing delta...
Processing output/dpli2\10_al_dpli_delta.npy...
Processing output/dpli2\10_fa_dpli_delta.npy...
Processing output/dpli2\11_al_dpli_delta.npy...
Processing output/dpli2\11_fa_dpli_delta.npy...
Processing output/dpli2\12_al_dpli_delta.npy...
Processing output/dpli2\12_fa_dpli_delta.npy...
Processing output/dpli2\13_al_dpli_delta.npy...
Processing output/dpli2\13_fa_dpli_delta.npy...
Processing output/dpli2\14_al_dpli_delta.npy...
Processing output/dpli2\14_fa_dpli_delta.npy...
Processing output/dpli2\16_al_dpli_delta.npy...
Processing output/dpli2\16_fa_dpli_delta.npy...
Processing output/dpli2\17_al_dpli_delta.npy...


In [5]:
f = "output/dpli/1_al_dpli_theta.npy"
data = read_file(f)
file_info = parse_filename(os.path.basename(f))
pId = file_info["pId"]
label = file_info["label"]
df_good = transform_data(data)
df_features = feature_extraction_lobe_level(df_good)


In [8]:
df_good

,source,target,value,source_lobe,target_lobe
273,caudalmiddlefrontal-lh,bankssts-rh,0.263514,Frontal-lh,Temporal-rh
344,caudalmiddlefrontal-rh,caudalmiddlefrontal-lh,0.250000,Frontal-rh,Frontal-lh
408,cuneus-lh,bankssts-lh,-0.277027,Occipital-lh,Temporal-lh
409,cuneus-lh,bankssts-rh,-0.216216,Occipital-lh,Temporal-rh
412,cuneus-lh,caudalmiddlefrontal-lh,-0.351351,Occipital-lh,Frontal-lh
...,...,...,...,...,...
4604,transversetemporal-rh,precentral-lh,0.270270,Temporal-rh,Frontal-lh
4609,transversetemporal-rh,rostralanteriorcingulate-rh,-0.378378,Temporal-rh,Frontal-rh
4612,transversetemporal-rh,superiorfrontal-lh,0.263514,Temporal-rh,Frontal-lh
4617,transversetemporal-rh,superiortemporal-rh,-0.358108,Temporal-rh,Temporal-rh


In [86]:
cols = d_stats_df['freq'].columns

In [103]:
d_stats_df['alpha'].count()

Frontal-lh_Frontal-lh      96
Frontal-lh_Frontal-rh      96
Frontal-lh_Insula-lh       95
Frontal-lh_Insula-rh       96
Frontal-lh_Occipital-lh    96
                           ..
Temporal-rh_Parietal-rh    96
Temporal-rh_Temporal-lh    96
Temporal-rh_Temporal-rh    96
pId                        96
label                      96
Length: 102, dtype: int64

In [120]:
from scipy import stats

df_stat = d_stats_df['delta'].sort_values(by=['label', 'pId'])
df_res = pd.DataFrame(columns=['con', 't', 'p'])

for col in cols:
    if col in ['pId', 'label']:
        continue

    df_stat[col] = df_stat[col].astype(float)
    x = df_stat[col].values[:48]
    y = df_stat[col].values[48:]

    mask_x = np.isnan(x)
    mask_y = np.isnan(y)
    x = x[~mask_x & ~mask_y]
    y = y[~mask_x & ~mask_y]

    t, p = stats.ttest_rel(x, y)
    # print(f"{col}: t = {t}, p = {p}")
    df_res = df_res.append({"con": col, "t": t, "p": p}, ignore_index=True)

df_res[df_res.p < 0.05]

,con,t,p
17,Frontal-rh_Temporal-lh,2.797889,0.007434
18,Frontal-rh_Temporal-rh,2.271089,0.027764
33,Occipital-lh_Occipital-rh,2.335458,0.023839
36,Occipital-lh_Temporal-lh,-2.873350,0.006079
50,Parietal-lh_Insula-lh,2.090450,0.042388
79,Temporal-rh_Frontal-rh,-2.305529,0.025598
